# Tensorflow 실습 3 : 딥러닝 template

## Custom 방식의 딥러닝 구현 

In [ ]:
import os
import tensorflow as tf
from time import localtime, time

### MNIST 데이터셋

In [ ]:
# Dataset Load
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ratio = 0.8

# 데이터를 0.8:0.2 비율로 traing, validation 데이터로 분리
x_train, x_valid = x_train[:int(len(x_train)*ratio)], x_train[int(len(x_train)*ratio):]
y_train, y_valid = y_train[:int(len(y_train)*ratio)], y_train[int(len(y_train)*ratio):]

print(x_train[0].shape)
print(x_train[0].dtype)
print(y_train[:5])
print(len(x_train))
print(len(x_valid))
print(len(x_test))

- tf.data.Dataset을 이용해서, dataset을 준비

In [ ]:
# batch size 설정
batch_size = 32

# training
# 불러온 데이터를 tf.data.Dataset의 형태로 변환
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# 이미지의 데이터 타입을 tf.float32로 변환 (기본 tf.keras.Model의 dtype이 tf.float32)
train_ds = train_ds.map(lambda x, y: (tf.cast(x, tf.float32), y))

# 학습 데이터에서는 shuffle을 이용함
train_ds = train_ds.shuffle(10000).batch(batch_size)

# validation
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
valid_ds = valid_ds.map(lambda x, y: (tf.cast(x, tf.float32), y))
valid_ds = valid_ds.batch(batch_size)

# test
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.float32), y))
test_ds = test_ds.batch(batch_size)

### Custom 모델 선언하기

- tf.keras.Model을 상속받아 class를 만들면, custom model을 만들수 있음
- tf.keras.models.Sequential()은 순차적으로 계산하는 것에 한정되어 복잡한 model 구조를 구성하기 어려움





In [ ]:
# tf.keras.Model을 상속 받아서 만든 class라는 것에 유의
class MyModel(tf.keras.Model):
    # __init__ 함수는 class를 처음 불러올 때, initialize하는 부분
    # model에서 사용할 layer를 설정함 (학습 및 저장하는 파라미터)
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
    # call 함수는 model에 input을 넣어 계산할 때 사용되는 함수
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.d1(x)
        # batch norm, dropout은 학습 시와 테스트 시의 연산이 다름
        x = self.batch_norm(x, training=training)
        x = self.dropout(x, training=training)
        return self.d2(x)

    def print_summary(self):
      x = tf.keras.Input(shape=(28,28))
      return print(tf.keras.Model(inputs=[x], outputs=self.call(x)).summary())

# MyModel()을 통해, 하나의 모델 instance를 생성
model = MyModel()
model.print_summary()

### 옵티마이저(optimizer), 손실 함수, 평가 metric 선택

In [ ]:
# 손실 함수 (loss function)
# 0부터 9까지 정수 값의 label을 분류하므로, SparseCategoricalCrossentropy를 사용
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# optimizer
optimizer = tf.keras.optimizers.Adam()

# 평가 metric은 loss와 accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

### tensorboard 준비

In [ ]:
# 학습과정 summary 저장 및 시각화를 위한 tensorboard log file
tb_path = os.path.join('tensorboard','tb-%02d%02d-%02d%02d' % tuple(localtime(time()))[1:5])
summary_writer = tf.summary.create_file_writer(tb_path)
tf.summary.trace_on(graph=True, profiler=True)
graph_save = True

### step 함수 정의
- 하나의 mini batch data를 이용하는 학습 연산 과정을 함수로 정의


In [ ]:
# @tf.function은 Autograph (일반적으로 속도 향상)
@tf.function
def train_step(images, labels, model, loss_metric, accuracy_metric):
    with tf.GradientTape() as tape:
        # model에 input을 넣어주면, call 함수가 실행 
        predictions = model(images, training=True)
        # loss_object를 이용하여 loss를 계산함
        loss = loss_object(labels, predictions)
    # 모델의 학습 가능한 파라미터들에 대한 gradient 값을 계산
    gradients = tape.gradient(loss, model.trainable_variables)
    # 모델의 각 파라미터에 대해, gradients를 적용
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # loss와 accuracy를 metric에 저장
    loss_metric(loss)
    accuracy_metric(labels, predictions)

- 하나의 mini batch data를 이용하여, validation 및 test하는 연산 과정을 함수로 정의
- 성능 평가를 위한 부분으로, gradient 계산이 필요 없음

In [ ]:
# @tf.function
def validation_and_test_step(images, labels, model, loss_metric, accuracy_metric):
    predictions = model(images, training=False)
    loss = loss_object(labels, predictions)
    loss_metric(loss)
    accuracy_metric(labels, predictions)

### 딥러닝 모델 학습하기

- 전체 학습 loop

In [ ]:
epochs = 5
t_step = 0
v_step = 0
# 각 epoch 마다 학습 및 validation을 진행

for epoch in range(epochs):

    # 학습
    for images, labels in train_ds:
        train_step(images, labels, model, train_loss, train_accuracy)

        # loss 값을 모든 step 마다 tensorboard log로 기록
        with summary_writer.as_default():
            tf.summary.scalar('train_loss', train_loss.result(), step=t_step)
        t_step += 1

        # 모델 연산 graph는 처음에 한번만 저장함
        if graph_save:
            with summary_writer.as_default():
                tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=tb_path)
            tf.summary.trace_off()
            graph_save = False

    # Validation
    for images, labels in valid_ds:
        validation_and_test_step(images, labels, model, valid_loss, valid_accuracy)

        with summary_writer.as_default():
            tf.summary.scalar('validation_loss', valid_loss.result(), step=v_step)
        v_step += 1

    # 각 epoch이 끝날 때마다, metric의 값을 출력
    template = 'Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Validation Loss: {:.3f}, Validation Accuracy: {:.3f}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result(),
                          valid_loss.result(),
                          valid_accuracy.result()))

    # 하나의 epoch이 끝나면, metric 값을 초기화 시켜줘야 함
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

In [ ]:
# Colab 환경에서 tensorboard를 사용하는 방법
%load_ext tensorboard
%tensorboard --logdir tensorboard/

### 학습된 모델 저장하기

In [ ]:
# 모델을 저장할 directory
ckpt_dir = 'ckpt'
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

# 모델을 저장할 path
save_path = os.path.join(ckpt_dir, 'run-%02d%02d-%02d%02d' % tuple(localtime(time()))[1:5])

# model.save_weights로 학습된 파라미터를 저장
model.save_weights(save_path, save_format='tf')

### 저장된 모델 불러오기

- 먼저 new_model이라는 instance를 새로 만들어서, 학습되지 않은 모델로 성능 평가

In [ ]:
# 기존 model 삭제
del model

# 새로운 모델
new_model = MyModel()

# test metrics
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 학습된 파라미터를 load하기 전의 모델
for images, labels in test_ds:
    validation_and_test_step(images, labels, new_model, test_loss, test_accuracy)

template = 'Before load trained model, Test Loss: {:.3f}, Test Accuracy: {:.3f}'
print(template.format(test_loss.result(),
                      test_accuracy.result()))


test_loss.reset_states()
test_accuracy.reset_states()


- 학습된 모델 파라미터 load 후 테스트

In [ ]:
# 학습된 모델 파라미터 load
new_model.load_weights(save_path)
for images, labels in test_ds:
    validation_and_test_step(images, labels, new_model, test_loss, test_accuracy)

# 학습된 모델로 테스트
template = 'Trained model, Test Loss: {:.3f}, Test Accuracy: {:.3f}'
print(template.format(test_loss.result(),
                      test_accuracy.result()))